In [1]:
%useLatestDescriptors
%use dataframe

In [5]:
// with resultats as (
// select r.author, r.comment, p.criterion_id , p.priority, p.comment as priority_comment
// ,min(p.priority) over (partition by r.author) as priority_min
// ,max(p.priority) over (partition by r.author) as priority_max
// from sogisurvey.responses as r
//         inner join sogisurvey.priorities as p
// on r.id = p.response_id
// )
// select author, comment, criterion_id, priority
// , 100 * (priority - priority_min) / (priority_max - priority_min) as normalized_priority
// , priority_comment
// from resultats
val responses = DataFrame.read("./results.csv")
responses.head()


author,comment,criterion_id,priority,normalized_priority,priority_comment
Antoine Du Bois,null,ambiance,83,79,null
Antoine Du Bois,null,localisation,31,15,null
Antoine Du Bois,null,progression,72,65,null
Antoine Du Bois,null,autonomie,33,18,null
Antoine Du Bois,null,domaine-metier,18,0,Curieux de découvrir tout type de cœu...


In [7]:
val authors = responses.distinct { author }
val responsesWithComment = responses
    .filter { comment?.isNotBlank() ?: false }
    .distinct { author and comment }
val html = buildString {
    append("<b>${authors.count()}</b> personnes ont répondu.")
    append("<ul>")
    responsesWithComment.forEach {
        append("<li><b>$author</b> a commenté : \"$comment\"</li>")
    }
    append("</ul>")
}
DISPLAY(HTML(html))

18 personnes ont répondu. Florian Ferrier a commenté : "Le choix de mission doit être fait avec des clients qui adhèrent à la philosophie de Sogilis. Et non comme "une ressource" supplémentaire (les autres ESN savent mieux vendre ça)." Gwennan Eliezer a commenté : "Bref, en gros j'ai besoin d'une bonne ambience de travail et d'un certain confort, le reste est secondaire" Jean-Baptiste Mille a commenté : "Autre critère : raison d'être & morale du client. Pas uniquement mercantile." Olivier Colin a commenté : "Autre critère : facilité d'accès à la documentation et aux référents fonctionnels"

In [9]:
%use kandy

In [10]:
//responses.add("color") { if (priority_comment !=null) "red" else "default" }.plot {
responses.plot {
    points {
        x(criterion_id) { axis.name = "Critère" }
        y(normalized_priority) {
            axis.name = "Importance"
            axis.breaks(listOf(0,50,100))
        }
//        color("color")
        tooltips(author, priority_comment)
        //symbol = Symbol.CROSS
        alpha = .4
    }
    layout {
        style {
            legend.position = LegendPosition.None
        }
    }
}
// TODO sort x axis

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xNJsDV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"normalized_priority":[79.0,15.0,65.0,18.0,0.0,75.0,100.0,14.0,57.0,15.0,58.0,57.0,18.0,20.0,68.0,74.0,64.0,89.0,94.0,0.0,0.0,0.0,77.0,26.0,27.0,75.0,76.0,50.0,0.0,100.0,50.0,0.0,39.0,100.0,28.0,13.0,0.0,43.0,100.0,4.0,44.0,92.0,43.0,43.0,72.0,69.0,4.0,43.0,62.0,0.0,22.0,19.0,22.0,100.0,54.0,82.0,79.0,41.0,54.0,88.0,59.0,18.0,100.0,100.0,14.0,0.0,40.0,0.0,0.0,34.0,0.0,0.0,100.0,68.0,42.0,0.0,0.0,32.0,2.0,66.0,84.0,76.0,34.0,74.0,0.0,24.0,88.0,42.0,64.0,64.0,84.0,91.0,100.0,0.0,0.0,25.0,55.0,100.0,100.0,0.0,0.0,74.0,74.0,100.0,86.0,24.0,14.0,14.0,30.0,64.0,100.0,38.0,85.0,2.0,0.0,8.0,64.0,69.0,54.0,47.0,100.0,55.0,47.0,8.0,8.0,30.0,17.0,26.0,91.0,98.0,36.0,0.0,50.0,66.0,40.0,100.0,93.0,96.0,88.0,68.0,28.0,86.0,80.0,30.0,0.0,73.0,72.0,92.0,62.0,62.0,60.0,97.0,77.0,100.0,7.0,36.0,36.0,78.0,36.0,78.0,11.0,46.0,51.0,1.0,0.0,94.0,2.0,86.0,0.0,100.0,77.0,58.0,40.0,67.0,0.0,0.0,40.0,32.0,65.0,0.0,0.0,100.0,70.0,81.0,19.0,84.0,38.0,0.0,50.0,0.0,0.0,77.0,41.0,14.0,63.0,67.0,0.0,73.0,96.0,100.0,50.0,33.0,86.0,70.0,21.0,77.0,42.0,59.0,14.0,41.0,100.0,50.0,50.0,56.0,56.0,79.0,79.0,79.0,62.0,79.0,56.0,0.0,100.0,27.0,80.0,82.0,20.0,74.0,25.0,74.0,100.0,100.0,57.0,93.0,93.0,54.0,18.0,76.0,0.0,28.0,57.0,57.0,25.0,62.0,18.0,87.0,0.0,43.0,84.0,100.0,27.0,77.0,63.0,68.0,40.0,40.0,21.0,50.0,27.0,29.0,59.0,100.0,72.0,88.0,100.0,100.0,8.0,48.0,100.0,66.0,0.0,36.0,79.0,100.0,79.0,0.0,100.0,48.0,78.0,80.0,100.0,79.0,6.0,48.0,52.0,48.0,48.0,16.0],
"author":["Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Antoine Du Bois","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Arthur Fanneau","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Aurélien LABROSSE","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Lyet","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Baptiste Robert","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Clara Gangneux","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Coralie RACHEX","Emmanuel Etievent","Emmanuel Etievent","Emmanuel Etievent","Emmanuel Etievent","Emmanuel Etievent","Emmanuel Etievent","Emmanuel Etieve